# 用于预训练BERT的数据集
为了预训练14.7节中实现的BERT模型，我们需要以理想的格式⽣成数据集，以便于两个预训练任务：遮蔽语
⾔模型和下⼀句预测。⼀⽅⾯，最初的BERT模型是在两个庞⼤的图书语料库和英语维基百科（参⻅14.7.5节
）的合集上预训练的，但它很难吸引这本书的⼤多数读者。另⼀⽅⾯，现成的预训练BERT模型可能不适合医
学等特定领域的应⽤。因此，在定制的数据集上对BERT进⾏预训练变得越来越流⾏。为了⽅便BERT预训练
的演⽰，我们使⽤了较小的语料库WikiText-2 [Merity et al., 2016] 。
与14.3节中⽤于预训练word2vec的PTB数据集相⽐，WikiText-2（1）保留了原来的标点符号，适合于下⼀句
预测；（2）保留了原来的⼤小写和数字；（3）⼤了⼀倍以上。

In [1]:
import os
import random
import torch
from d2l import torch as d2l

在WikiText-2数据集中，每⾏代表⼀个段落，其中在任意标点符号及其前⾯的词元之间插⼊空格。保留⾄少有两句话的段落。为了简单起⻅，我们仅使⽤句号作为分隔符来拆分句⼦。

In [2]:
#@save
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')

#@save
def _read_wiki(data_dir): # 下载下来，并load进来
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    
    # ⼤写字⺟转换为⼩写字⺟
    paragraphs = [line.strip().lower().split(' . ')
                    for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

# 为预训练任务定义辅助函数
_get_next_sentence函数⽣成⼆分类任务的训练样本。

In [3]:
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5: # 50%的可能性就是相邻的句子
        is_next = True
    else: # 50%可能性，不是相邻的句子
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    
    return sentence, next_sentence, is_next

# 下⾯的函数通过调⽤paragraph函数从输⼊_get_next_sentence⽣成⽤于下⼀句预测的训练样本。这
# ⾥paragraph是句⼦列表，其中每个句⼦都是词元列表。⾃变量max_len指定预训练期间的BERT输⼊序列
# 的最⼤⻓度。
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(paragraph[i], paragraph[i + 1], paragraphs)
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

# ⽣成遮蔽语⾔模型任务的数据
为了从BERT输⼊序列⽣成遮蔽语⾔模型的训练样本，我们定义了以下_replace_mlm_tokens函数。在其
输⼊中，tokens是表⽰BERT输⼊序列的词元的列表，candidate_pred_positions是不包括特殊词元
的BERT输⼊序列的词元索引的列表（特殊词元在遮蔽语⾔模型任务中不被预测），以及num_mlm_preds指
⽰预测的数量（选择15%要预测的随机词元）。在14.7.5节中定义遮蔽语⾔模型任务之后，在每个预测位置，
输⼊可以由特殊的“掩码”词元或随机词元替换，或者保持不变。最后，该函数返回可能替换后的输⼊词元、
发⽣预测的词元索引和这些预测的标签。

In [4]:
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab):
    """随机遮蔽一些词"""
    # 为遮蔽语⾔模型的输⼊创建新的词元副本，其中输⼊可能包含替换的“<mask>”或随机词元
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # 打乱后⽤于在遮蔽语⾔模型任务中获取15%的随机词元进⾏预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80%的时间：将词替换为“<mask>”词元
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10%的时间：保持词不变
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10%的时间：⽤随机词替换该词
            else:
                masked_token = random.randint(0, len(vocab) - 1)
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels

"""
通过调⽤前述的_replace_mlm_tokens函数，以下函数将BERT输⼊序列（tokens）作为输⼊，并返回输
⼊词元的索引（在前面描述的可能的词元替换之后）、发⽣预测的词元索引以及这些预测的标签索引。
"""

def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    # `tokens`是⼀个字符串列表
    for i, token in enumerate(tokens):
        # 在遮蔽语⾔模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    # 遮蔽语⾔模型任务中预测15%的随机词元
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

# 将⽂本转换为预训练数据集
现在我们⼏乎准备好为BERT预训练定制⼀个Dataset类。在此之前， 我们仍然需要定义辅助函
数_pad_bert_inputs来将特殊的“<mask>”词元附加到输⼊。它的参数examples包含来⾃两个预训练
任务的辅助函数_get_nsp_data_from_paragraph和_get_mlm_data_from_tokens的输出。

In [6]:
def _pad_bert_inputs(examples, max_len, vocab):
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    for (token_ids, pred_positions, mlm_pred_label_ids, segments,
         is_next) in examples:
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (
            max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (
            max_len - len(segments)), dtype=torch.long))
        # `valid_lens` 不包括'<pad>'的计数
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (
            max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # 填充词元的预测将通过乘以0权重在损失中过滤掉
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (
                max_num_mlm_preds - len(pred_positions)),
                dtype=torch.float32))
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (
            max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

# The WikiText-2 dataset for pretraining BERT

将⽤于⽣成两个预训练任务的训练样本的辅助函数和⽤于填充输⼊的辅助函数放在⼀起，我们定义以
下_WikiTextDataset类为⽤于预训练BERT的WikiText-2数据集。通过实现__getitem__函数，我们可
以任意访问WikiText-2语料库的⼀对句⼦⽣成的预训练样本（遮蔽语⾔模型和下⼀句预测）样本。
最初的BERT模型使⽤词表⼤小为30000 的WordPiece嵌⼊ 。WordPiece的词元化⽅法是
对subsec_Byte_Pair_Encoding 中原有的字节对编码算法稍作修改。为简单起⻅，我们使⽤d2l.
tokenize函数进⾏词元化。出现次数少于5次的不频繁词元将被过滤掉。

In [8]:
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # 输⼊`paragraphs[i]`是代表段落的句⼦字符串列表；⽽输出`paragraphs[i]`是代表段落的句⼦列表，其中每个句⼦都是词元列表
        paragraphs = [d2l.tokenize(
            paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])
        # 获取下⼀句⼦预测任务的数据
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))
        # 获取遮蔽语⾔模型任务的数据
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                      + (segments, is_next))
                     for tokens, segments, is_next in examples]
        # 填充输⼊
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

# 通过使⽤_read_wiki函数和_WikiTextDataset类，我们定义了下⾯的load_data_wiki来下载并⽣
成WikiText-2数据集，并从中⽣成预训练样本。
通过使⽤_read_wiki函数和_WikiTextDataset类，我们定义了下⾯的load_data_wiki来下载并⽣
成WikiText-2数据集，并从中⽣成预训练样本。

In [9]:
def load_data_wiki(batch_size, max_len):
    """Load the WikiText-2 dataset."""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                        shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

# Print out the shapes of a minibatch of BERT pretraining examples
将批量⼤小设置为512，将BERT输⼊序列的最⼤⻓度设置为64，我们打印出小批量的BERT预训练样本的形
状。注意，在每个BERT输⼊序列中，为遮蔽语⾔模型任务预测10（64 * 0.15）个位置

In [20]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
     mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,
          nsp_y.shape)
    break

UnicodeDecodeError: 'gbk' codec can't decode byte 0xae in position 96: illegal multibyte sequence

In [ ]:
len(vocab)